In [ ]:
%matplotlib notebook
from time import sleep
import matplotlib.pyplot as plt
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#decide interval and no of pages
a=int(input("From : "))
b=int(input("To : "))
c=int(input("No of pages to be scraped per year : "))

In [ ]:
pages = [str(i) for i in range(1,c+1)]
years_url = [str(i) for i in range(a,b+1)]

In [ ]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

In [ ]:
# For every year in the interval
for year_url in years_url:
    
    # For every page
    for page in pages:
        
        # Make a get request
        response = requests.get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page)
        
        # Pause the loop
        sleep(5)
        
        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:
                
                # Scrape the name
                name = container.h3.a.text
                names.append(name)
                
                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

In [ ]:
movie_ratings = pd.DataFrame({'movie': names,'year': years,'imdb': imdb_ratings,'metascore': metascores,'votes': votes})

#reorder column
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes']]

In [ ]:
#convert year to int
movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(int)

In [ ]:
#multiply imdb by 10
movie_ratings['n_imdb'] = movie_ratings['imdb'] * 10

In [ ]:
#save dataset
movie_ratings.to_csv('dataset.csv')

In [ ]:
#Plot graphs
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (11,4))
ax1, ax2, ax3 = fig.axes

ax1.hist(movie_ratings['imdb'], bins = 10, range = (0,10))
ax1.set_title('IMDB rating')

ax2.hist(movie_ratings['metascore'], bins = 10, range = (0,100))
ax2.set_title('Metascore')

ax3.hist(movie_ratings['n_imdb'], bins = 10, range = (0,100), histtype = 'step')
ax3.hist(movie_ratings['metascore'], bins = 10, range = (0,100), histtype = 'step')
ax3.legend(loc = 'upper left')
ax3.set_title('The Two Normalized Distributions')

for ax in fig.axes:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.show()